# NEAR Project – Attention-Aware Interpretation

## Overview

This notebook implements the **LLM-based Attention-Aware Interpretation layer** of the NEAR Project.

It consumes:

* Pre-generated gaze heatmaps
* Extracted AOI crops
* Windowed frame outputs from the Crop & Generator pipeline

It produces:

* Task-aware LLM summaries
* Structured demo-ready output folders
* Composite visualization images
* JSON response files
* Streamlit demo assets

This notebook does **NOT** generate gaze heatmaps.
Heatmap and crop generation are handled by the Crop & Generator notebook.

# Gaze Heatmap & Crops Pipeline

## Overview

This notebook processes **eye-tracking gaze data** from the NEAR Experiment Design dataset. For each user folder it:

1. **Loads gaze data** (from Pupil exports CSV or pldata).
2. **Splits the recording into time windows** (e.g. 3 s).
3. **For each window:** grabs a video frame, builds a **heatmap** (2D histogram + Gaussian blur + jet colormap overlay), saves full-frame heatmap and source images, computes a **region of interest** from the heatmap, and saves **cropped** versions of both.
4. **Builds an MP4** from the heatmap frames for easy playback.


Generates:

* Windowed video frames
* Gaze heatmaps
* AOI crops
* Heatmap MP4 videos

---

# 1. Raw Input Structure

Base folder: `PilotData_V1_10232025/`

Per subject-task folder:

```
[Subject_Task]/
├── world.mp4
├── world_timestamps.npy
└── exports/
    └── 000/
        ├── gaze_positions.csv
        ├── fixations.csv
        └── pupil_positions.csv
```

---

# 2. Processing Steps

1. Segment recording into fixed windows (default: 3 seconds)
2. Extract mid-window frame
3. Generate heatmap:

   * 2D histogram
   * Gaussian smoothing
   * Jet colormap overlay
4. Save:

   * Source frame
   * Heatmap
5. Compute AOI region from heatmap density
6. Save cropped versions
7. Compile heatmap frames into MP4

---

# 3. Output Structure

Output folder: `BASE_OUTPUT_PATH/<user_name>/frames/`

Per subject-task folder:
```
Ayu_1/
└── frames/
    ├── src_000-003s.png         — source video frame at mid-window.
    ├── heat_000-003s.png        — heatmap overlay (same style as Data_Analysis original).
    ├── src_000-003s_crop.png    — crops around the gaze-dense region.
    ├── heat_000-003s_crop.png   — crops around the gaze-dense region.
    └── Ayu_1_heatmap.mp4        — video of heatmap frames in time order.
```

Heatmap method matches the original Data_Analysis pipeline (2D histogram, Gaussian smoothing, jet colormap, matplotlib figure at 200 dpi).

## 1. Mount Google Drive

Mount Drive so we can read the source dataset and write outputs. Run this cell first.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Imports

Libraries for gaze loading (including Pupil pldata via msgpack), video (OpenCV), heatmap (matplotlib, scipy), and MP4 writing (imageio, PIL).  
If you get an import error, run: `%pip install msgpack scipy pillow imageio imageio-ffmpeg`

In [ ]:
# Import required libraries (heatmap from Data_Analysis_old_version)
# If needed: %pip install msgpack scipy pillow imageio imageio-ffmpeg
import os
import glob
import re
import msgpack
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from collections import Counter
from PIL import Image
import imageio.v2 as imageio

## 3. Paths and user list

- **BASE_SOURCE_PATH:** folder containing one subfolder per user (e.g. `Ayu_1`, `AT_1`), each with `world.mp4` and `exports/` (or pldata).
- **BASE_OUTPUT_PATH:** where to write per-user `frames/` and heatmap MP4s.  
We list all non-hidden subfolders of the source path as user folders (excluding `1_Data_Analysis`).

In [ ]:
# Where to read data (one subfolder per user: Ayu_1, AT_1, ...) and where to write results
BASE_SOURCE_PATH = "/content/drive/MyDrive/NEAR_Experiment_Design/PilotData_V1_10232025"
BASE_OUTPUT_PATH = "/content/drive/MyDrive/NEAR_Experiment_Design_Output"
os.makedirs(BASE_OUTPUT_PATH, exist_ok=True)

# Discover all user folders (each must contain world.mp4 and exports/ or pldata)
user_folders = sorted([f for f in os.listdir(BASE_SOURCE_PATH)
                       if os.path.isdir(os.path.join(BASE_SOURCE_PATH, f))
                       and not f.startswith('.')
                       and f not in ['1_Data_Analysis']])  # Skip non-user folders
print(f"Found {len(user_folders)} user folders:")
for folder in user_folders[:10]:  # Show first 10
    print(f"  - {folder}")
if len(user_folders) > 10:
    print(f"  ... and {len(user_folders) - 10} more")

Found 26 user folders:
  - AT_1
  - AT_2
  - AT_3_1
  - AT_3_2
  - Ayu_1
  - Ayu_2
  - Ayu_3
  - JC_1
  - JC_2
  - JC_3_1
  ... and 16 more


## 4. Helper functions

- **Gaze:** `load_gaze_dataframe(task_dir)` — loads from `exports/**/gaze_positions.csv` or, if missing, from Pupil `.pldata`; returns DataFrame with `timestamp`, `norm_pos_x`, `norm_pos_y`, `confidence`.
- **Video:** `open_world_video(task_dir)`, `grab_frame_at_time(cap, fps, t_sec)` — open `world.mp4` and seek to a time.
- **Heatmap:** `compute_heat(df_window, h, w)` — 2D histogram of gaze (Y flipped) + Gaussian blur; used for crop bbox.  
  `plot_and_save_heatmap(bg_rgb, df_window, path)` — same heatmap drawn with matplotlib (jet, alpha 0.5), saved as PNG (200 dpi).
- **Crop:** `bbox_from_heatmap_only(heat, pad, thresh)` — bounding box from heat above threshold.
- **MP4:** `sort_by_window(files)` (by start time in filename), `to_even_size`, `make_mp4_from_folder(folder, pattern, out_mp4, fps)` — build MP4 from heatmap PNGs.

## 5. Main processing function

`process_user_folder(user_name, source_base_path, output_base_path, ...)` does everything for one user:

1. Load gaze and open video; align gaze time to relative seconds (`t_rel`).
2. For each time window: grab frame at window mid-time; save source PNG; build and save heatmap PNG; compute heat array and bbox; save source and heatmap crops (heatmap crop uses bbox scaled to figure size 2000×1200).
3. Build `<user_name>_heatmap.mp4` from all `heat_*-*s.png` frames.

Parameters: `interval_sec` (window length in seconds), `pad` (pixels around bbox for crop), `sample_windows` (cap number of windows if set).

In [ ]:
# Heatmap pipeline from Data_Analysis_old_version (Ayu_1_heatmap.mp4 style)
def load_pldata_file(directory, topic):
    ts_file = os.path.join(directory, f"{topic}_timestamps.npy")
    mp_file = os.path.join(directory, f"{topic}.pldata")
    data_list = []
    if not (os.path.exists(ts_file) and os.path.exists(mp_file)):
        return [], np.array([])
    timestamps = np.load(ts_file)
    with open(mp_file, "rb") as fh:
        unpacker = msgpack.Unpacker(fh, raw=False, use_list=False)
        for tpc, payload in unpacker:
            datum = msgpack.unpackb(payload, raw=False, use_list=False)
            data_list.append(datum)
    return data_list, timestamps

def pldata_gaze_to_dataframe(directory):
    def _normalize_to_unit(series):
        v = series.astype(float)
        vmin, vmax = float(v.min()), float(v.max())
        rng = vmax - vmin
        if (vmin < 0) or (vmax > 1):
            if rng <= 1.2:
                v = (v - vmin) / max(rng, 1e-9)
            elif rng <= 2.2 and (vmin >= -1.1) and (vmax <= 1.1):
                v = (v + 1.0) / 2.0
            else:
                v = v.clip(0, 1)
        return v
    data_list, ts = load_pldata_file(directory, "gaze")
    if len(ts) == 0:
        return pd.DataFrame()
    rows = []
    for d, tt in zip(data_list, ts):
        row = {"timestamp": float(d.get("timestamp", tt))}
        if "norm_pos" in d:
            nx, ny = d["norm_pos"][0], d["norm_pos"][1]
        else:
            nx, ny = d.get("norm_pos_x", np.nan), d.get("norm_pos_y", np.nan)
        row["norm_pos_x"] = float(nx) if nx is not None else np.nan
        row["norm_pos_y"] = float(ny) if ny is not None else np.nan
        row["confidence"] = float(d.get("confidence", np.nan))
        rows.append(row)
    df = pd.DataFrame(rows).dropna(subset=["norm_pos_x", "norm_pos_y"])
    if (df["norm_pos_x"].min() < 0) or (df["norm_pos_x"].max() > 1) or (df["norm_pos_y"].min() < 0) or (df["norm_pos_y"].max() > 1):
        df["norm_pos_x"] = _normalize_to_unit(df["norm_pos_x"])
        df["norm_pos_y"] = _normalize_to_unit(df["norm_pos_y"])
    return df

def exports_csv_to_dataframe(exports_dir):
    candidates = glob.glob(os.path.join(exports_dir, "**", "gaze_positions.csv"), recursive=True)
    if not candidates:
        return pd.DataFrame()
    path = candidates[0]
    df = pd.read_csv(path)
    ts_col = next((c for c in ["gaze_timestamp", "timestamp", "world_timestamp", "time", "system_time"] if c in df.columns), None)
    if ts_col is None:
        return pd.DataFrame()
    if {"norm_pos_x", "norm_pos_y"}.issubset(df.columns):
        out = pd.DataFrame()
        out["timestamp"] = df[ts_col].astype(float)
        out["norm_pos_x"] = df["norm_pos_x"].astype(float)
        out["norm_pos_y"] = df["norm_pos_y"].astype(float)
        out["confidence"] = df["confidence"].astype(float) if "confidence" in df.columns else np.nan
        return out.sort_values("timestamp").reset_index(drop=True)
    if {"gaze_point_2d_x", "gaze_point_2d_y"}.issubset(df.columns):
        out = pd.DataFrame()
        out["timestamp"] = df[ts_col].astype(float)
        out["norm_pos_x"] = df["gaze_point_2d_x"].astype(float)
        out["norm_pos_y"] = df["gaze_point_2d_y"].astype(float)
        out["confidence"] = df["confidence"].astype(float) if "confidence" in df.columns else np.nan
        return out.sort_values("timestamp").reset_index(drop=True)
    return pd.DataFrame()

def load_gaze_dataframe(task_dir):
    exports_dir = os.path.join(task_dir, "exports")
    df_csv = exports_csv_to_dataframe(exports_dir)
    if not df_csv.empty:
        return df_csv
    df_pl = pldata_gaze_to_dataframe(task_dir)
    if not df_pl.empty:
        return df_pl
    raise FileNotFoundError("Could not load gaze data from exports CSV or pldata.")

def open_world_video(task_dir):
    mp4_path = os.path.join(task_dir, "world.mp4")
    if not os.path.exists(mp4_path):
        cand = glob.glob(os.path.join(task_dir, "**", "world.mp4"), recursive=True)
        if cand:
            mp4_path = cand[0]
    cap = cv2.VideoCapture(mp4_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"Cannot open video at: {mp4_path}")
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps if fps > 0 else 0.0
    return cap, fps, frame_count, duration, width, height

def grab_frame_at_time(cap, fps, t_sec):
    frame_idx = int(round(t_sec * fps))
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ok, frame = cap.read()
    return frame if ok else None

def compute_heat(df_window, h, w, blur_ratio=0.05):
    gx = df_window["norm_pos_x"].to_numpy()
    gy = 1.0 - df_window["norm_pos_y"].to_numpy()
    hist, _, _ = np.histogram2d(gy, gx, bins=[h, w], range=[[0, 1], [0, 1]])
    fh = max(1, (int(blur_ratio * h) // 2) * 2 + 1)
    fw = max(1, (int(blur_ratio * w) // 2) * 2 + 1)
    heat = gaussian_filter(hist, sigma=(fh, fw), order=0)
    return heat

def plot_and_save_heatmap(bg_image_rgb, df_window, out_path_png, blur_ratio=0.05):
    os.makedirs(os.path.dirname(out_path_png), exist_ok=True)
    h, w = bg_image_rgb.shape[:2]
    gx = df_window["norm_pos_x"].to_numpy()
    gy = 1.0 - df_window["norm_pos_y"].to_numpy()
    hist, _, _ = np.histogram2d(gy, gx, bins=[h, w], range=[[0, 1], [0, 1]])
    fh = max(1, (int(blur_ratio * h) // 2) * 2 + 1)
    fw = max(1, (int(blur_ratio * w) // 2) * 2 + 1)
    heat = gaussian_filter(hist, sigma=(fh, fw), order=0)
    plt.figure(figsize=(10, 6))
    plt.imshow(bg_image_rgb)
    plt.imshow(heat, cmap="jet", alpha=0.5)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.savefig(out_path_png, dpi=200, bbox_inches=None, pad_inches=0)
    plt.close()

def bbox_from_heatmap_only(hmap, pad=0, thresh=0.5):
    if hmap.size == 0 or hmap.max() <= 0:
        return None
    hmap_n = hmap / hmap.max()
    mask = hmap_n > thresh
    ys, xs = np.where(mask)
    if len(xs) == 0:
        return None
    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    x0, y0 = max(0, x0 - pad), max(0, y0 - pad)
    x1, y1 = min(hmap.shape[1], x1 + pad), min(hmap.shape[0], y1 + pad)
    return x0, y0, x1, y1

def sort_by_window(files):
    def key_fn(p):
        m = re.search(r"_(\d+)-\d+s", os.path.basename(p))
        return int(m.group(1)) if m else 10**9
    return sorted(files, key=key_fn)

def to_even_size(size):
    w, h = size
    w, h = w - (w % 2), h - (h % 2)
    return (max(2, w), max(2, h))

def make_mp4_from_folder(folder, pattern, out_mp4_path, fps=2):
    files = sort_by_window(glob.glob(os.path.join(folder, pattern)))
    if not files:
        return
    sizes = [Image.open(p).size for p in files]
    common_size = Counter(sizes).most_common(1)[0][0]
    target_size = to_even_size(common_size)
    with imageio.get_writer(out_mp4_path, fps=fps, codec="libx264", quality=8, macro_block_size=None) as writer:
        for p in files:
            im = Image.open(p).convert("RGB")
            if im.size != target_size:
                im = im.resize(target_size, Image.LANCZOS)
            writer.append_data(np.array(im))
    print(f"[video] Saved: {out_mp4_path} ({len(files)} frames)")



In [ ]:
def process_user_folder(user_name, source_base_path, output_base_path,
                        interval_sec=3.0, pad=20, alpha=0.5, sample_windows=None):
    """Process one user: load gaze + video, build heatmaps per window, save crops and MP4."""
    user_source_path = os.path.join(source_base_path, user_name)
    frames_dir = os.path.join(output_base_path, user_name, "frames")
    os.makedirs(frames_dir, exist_ok=True)

    if not os.path.isdir(user_source_path):
        print(f"⚠️  {user_name}: folder not found")
        return False

    print(f"📊 Processing {user_name}...")
    try:
        gaze_df = load_gaze_dataframe(user_source_path)
    except Exception as e:
        print(f"❌ {user_name}: Failed to load gaze - {e}")
        return False
    try:
        cap, fps, frame_count, duration, W, H = open_world_video(user_source_path)
    except Exception as e:
        print(f"❌ {user_name}: Failed to open video - {e}")
        return False

    # Align gaze time to 0 and compute number of windows
    t_min = float(gaze_df["timestamp"].min())
    gaze_df = gaze_df.assign(t_rel=gaze_df["timestamp"] - t_min)
    max_duration = min(duration, float(gaze_df["t_rel"].max()))
    num_windows = int(max_duration // interval_sec) + 1
    if sample_windows is not None:
        num_windows = min(num_windows, sample_windows)

    processed = 0
    os.makedirs(frames_dir, exist_ok=True)
    for k in range(num_windows):
        start_t = k * interval_sec
        end_t = min((k + 1) * interval_sec, max_duration)
        if end_t <= start_t + 1e-6:
            continue
        win_mask = (gaze_df["t_rel"] >= start_t) & (gaze_df["t_rel"] < end_t)
        df_win = gaze_df.loc[win_mask]
        if df_win.empty:
            continue

        mid_t = 0.5 * (start_t + end_t)
        frame_bgr = grab_frame_at_time(cap, fps, mid_t)
        if frame_bgr is None:
            continue
        frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

        win_tag = f"{int(start_t):03d}-{int(end_t):03d}s"
        heat_path = os.path.join(frames_dir, f"heat_{win_tag}.png")
        src_path = os.path.join(frames_dir, f"src_{win_tag}.png")

        cv2.imwrite(src_path, frame_bgr)
        plot_and_save_heatmap(frame_rgb, df_win, heat_path)

        heat = compute_heat(df_win, H, W)
        bbox = bbox_from_heatmap_only(heat, pad=pad)
        if bbox is not None:
            x0, y0, x1, y1 = bbox
            crop_src = frame_bgr[y0:y1, x0:x1]
            cv2.imwrite(os.path.join(frames_dir, f"src_{win_tag}_crop.png"), crop_src)
            saved_heat = cv2.imread(heat_path)
            if saved_heat is not None:
                # Heatmap PNG is 2000x1200 (matplotlib fig 10x6 @ 200 dpi); scale bbox to crop it
                H_fig, W_fig = 1200, 2000
                sx, sy = W_fig / W, H_fig / H
                x0f, x1f = int(x0 * sx), int(x1 * sx)
                y0f, y1f = int(y0 * sy), int(y1 * sy)
                crop_heat = saved_heat[y0f:y1f, x0f:x1f]
                cv2.imwrite(os.path.join(frames_dir, f"heat_{win_tag}_crop.png"), crop_heat)
            processed += 1

    cap.release()

    heat_mp4_path = os.path.join(frames_dir, f"{user_name}_heatmap.mp4")
    make_mp4_from_folder(frames_dir, "heat_*-*s.png", heat_mp4_path, fps=2)

    print(f"✅ {user_name}: Processed {processed} intervals, heatmap MP4: {heat_mp4_path}")
    return True



## 6. Run pipeline

Set **SINGLE_USER** to a folder name (e.g. `"Ayu_1"`) to process only that user, or set to **`None`** to process **all users** in `user_folders`.  
Outputs go under `BASE_OUTPUT_PATH/<user_name>/frames/`.

In [ ]:
# --- Configuration ---
INTERVAL_SEC = 3.0      # Time window length in seconds (e.g. 3 = one heatmap every 3 s)
PAD = 20               # Extra pixels around heatmap ROI for crop
SAMPLE_WINDOWS = None  # None = process full recording; set to int to limit number of windows per user

# Process one user (set name) or all users (set to None)
SINGLE_USER = None     # e.g. None for all users, or "Ayu_1" to run only that folder

# --- Run ---
users_to_run = [SINGLE_USER] if SINGLE_USER else user_folders
print(f"Processing {len(users_to_run)} user(s): {users_to_run[:5]}{'...' if len(users_to_run) > 5 else ''}\n")

successful = 0
failed = 0
for user in users_to_run:
    try:
        result = process_user_folder(user, BASE_SOURCE_PATH, BASE_OUTPUT_PATH,
                                     interval_sec=INTERVAL_SEC, pad=PAD, alpha=0.5,
                                     sample_windows=SAMPLE_WINDOWS)
        if result:
            successful += 1
        else:
            failed += 1
    except Exception as e:
        print(f"❌ {user}: {e}")
        failed += 1

print(f"\n✅ Done: {successful} ok, {failed} failed. Outputs under: {BASE_OUTPUT_PATH}")

Processing 26 user(s): ['AT_1', 'AT_2', 'AT_3_1', 'AT_3_2', 'Ayu_1']...

📊 Processing AT_1...
[video] Saved: /content/drive/MyDrive/NEAR_Experiment_Design_Output/AT_1/frames/AT_1_heatmap.mp4 (24 frames)
✅ AT_1: Processed 24 intervals, heatmap MP4: /content/drive/MyDrive/NEAR_Experiment_Design_Output/AT_1/frames/AT_1_heatmap.mp4
📊 Processing AT_2...
[video] Saved: /content/drive/MyDrive/NEAR_Experiment_Design_Output/AT_2/frames/AT_2_heatmap.mp4 (10 frames)
✅ AT_2: Processed 10 intervals, heatmap MP4: /content/drive/MyDrive/NEAR_Experiment_Design_Output/AT_2/frames/AT_2_heatmap.mp4
📊 Processing AT_3_1...
[video] Saved: /content/drive/MyDrive/NEAR_Experiment_Design_Output/AT_3_1/frames/AT_3_1_heatmap.mp4 (7 frames)
✅ AT_3_1: Processed 7 intervals, heatmap MP4: /content/drive/MyDrive/NEAR_Experiment_Design_Output/AT_3_1/frames/AT_3_1_heatmap.mp4
📊 Processing AT_3_2...
[video] Saved: /content/drive/MyDrive/NEAR_Experiment_Design_Output/AT_3_2/frames/AT_3_2_heatmap.mp4 (15 frames)
✅ AT_3_2: 

In [ ]:
# To process only one user (e.g. for testing), set SINGLE_USER in the cell above:
#   SINGLE_USER = "Ayu_1"
# Then re-run the "Run pipeline" cell. To process everyone again, set:
#   SINGLE_USER = None

## 7. Check outputs

Summarizes how many frames and crops were written per user. Run after the pipeline has finished.

In [ ]:
def check_output_files(user_name):
    """Print a short summary of generated files for one user."""
    frames_dir = os.path.join(BASE_OUTPUT_PATH, user_name, "frames")
    if not os.path.exists(frames_dir):
        print(f"  {user_name}: no output")
        return
    files = sorted(os.listdir(frames_dir))
    src_full = [f for f in files if f.startswith("src_") and "_crop" not in f]
    heat_full = [f for f in files if f.startswith("heat_") and "_crop" not in f and not f.endswith(".mp4")]
    heat_crop = [f for f in files if "heat_" in f and "_crop" in f]
    mp4s = [f for f in files if f.endswith(".mp4")]
    print(f"  {user_name}: {len(src_full)} frames, {len(heat_crop)} crops, MP4: {len(mp4s)}")

# Summary for every user that has output (after running the pipeline)
print("Output summary (users with frames/ folder):")
for user in user_folders:
    check_output_files(user)


📁 Output files for Ayu_1:
  Source full frames: 20
  Source crops:       20
  Heat full frames:   20
  Heat crops:         20
  Total files:        81


# Automated Attention-Aware Output Generator
This iterates through NEAR project folders, analyzes gaze data with Gemini, and saves composite results.

---

## Input File Structure

Folder: `BASE_OUTPUT_PATH/<user_name>/frames/`

Per subject-task folder:
```
Ayu_1/
└── frames/
    ├── src_000-003s.png         — source video frame at mid-window.
    ├── heat_000-003s.png        — heatmap overlay (same style as Data_Analysis original).
    ├── src_000-003s_crop.png    — crops around the gaze-dense region.
    ├── heat_000-003s_crop.png   — crops around the gaze-dense region.
    └── Ayu_1_heatmap.mp4        — video of heatmap frames in time order.
```

## Output File Structure
All generated visual outputs are saved under:
```/content/drive/My Drive/AI_event_demo/Ayush_only/demo_data```

Each task (`describe`, `compare`, `recall`) has its own subdirectory:

```
demo_data/
  describe/                  # Task 1
    heatmap.mp4              # Generated gaze heatmap video
    merged/                  # Merged visualization image
      001.png
      002.png
      ...
    Original_image/          # Original stimulus frames
      001.png
      002.png
      ...
    aois/                    # AOI overlay images
      001.png
      002.png
      ...
    responses.json           # LLM-generated textual responses
  compare/                   # Task 2
    Heatmap.mp4
    merged/       
    Original_image/         
    aois/              
    responses.json         
  recall/                    # Task 3, 3_1, 3_2
    heatmap.mp4
    merged/
    Original_image/            
    aois/                    
    responses.json          
```

In [ ]:
!pip install -q openai

## Configuration and helper functions

In [5]:
import os
import re
import glob
import json
import shutil
import base64
from typing import Optional, Tuple, Dict

from tqdm.auto import tqdm
from PIL import Image, ImageDraw, ImageFont
from openai import OpenAI
from google.colab import drive, userdata

# ============================================================
# 1) AUTHENTICATION
# ============================================================
OPENAI_API_KEY = userdata.get("OPENAI_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

drive.mount("/content/drive")

# ============================================================
# 2) PATH CONFIG
# ============================================================
BASE_PATH = "/content/drive/My Drive/AI_event_demo/NEAR_Experiment_Design_Output/"
OUTPUT_ROOT = "/content/drive/My Drive/AI_event_demo/Ayush_only"
DEMO_PATH = "/content/drive/My Drive/AI_event_demo"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

TASK_GROUPS = [
    ["Ayu_1", "Ayu_2", "Ayu_3"],
]
ALL_FOLDERS = [f for g in TASK_GROUPS for f in g]

# Output structure root
DEMO_ROOT = os.path.join(DEMO_PATH, "demo_data")
os.makedirs(DEMO_ROOT, exist_ok=True)

# ============================================================
# 3) FONT SETUP
# ============================================================
def load_fonts():
    mono_regular = "/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf"
    mono_bold    = "/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf"

    if os.path.exists(mono_regular) and os.path.exists(mono_bold):
        header = ImageFont.truetype(mono_bold, 18)
        label  = ImageFont.truetype(mono_bold, 13)
        body   = ImageFont.truetype(mono_regular, 13)
        print("[Font] Using LiberationMono")
        return header, label, body

    print("[Font] LiberationMono not found. Falling back.")
    return ImageFont.load_default(), ImageFont.load_default(), ImageFont.load_default()

FONT_HEADER, FONT_LABEL, FONT_BODY = load_fonts()

# ============================================================
# 4) HELPERS
# ============================================================
def encode_image(path: str) -> str:
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def parse_task_type(folder_name: str) -> str:
    name = folder_name.replace("_simple", "")
    if "_3_1" in name:
        return "task3_1"
    if "_3_2" in name:
        return "task3_2"
    if name.endswith("_1"):
        return "task1"
    if name.endswith("_2"):
        return "task2"
    if name.endswith("_3"):
        return "task3"
    return "unknown"

def get_group_name(folder: str) -> str:
    return folder.split("_")[0]

def get_reference_folder(folder: str, suffix: str) -> str:
    return f"{get_group_name(folder)}{suffix}"

def prefer_crop(frames_dir: str, idx: str) -> Optional[str]:
    """
    Prefer src_{idx}_crop.png, else heat_{idx}_crop.png.
    idx example: '000-003s'
    """
    src_crop = os.path.join(frames_dir, f"src_{idx}_crop.png")
    heat_crop = os.path.join(frames_dir, f"heat_{idx}_crop.png")
    if os.path.exists(src_crop):
        return src_crop
    if os.path.exists(heat_crop):
        return heat_crop
    return None

def sanitize_model_text(text: str) -> str:
    if not text:
        return ""
    text = re.sub(r"^\s*#{1,6}\s*", "", text, flags=re.MULTILINE)
    text = text.replace("**", "").replace("*", "")
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def parse_src_filename(src_path: str) -> Tuple[str, int, int]:
    """
    Enforce: src_000-003s.png
    Returns:
      idx: '000-003s'
      start: 0
      end: 3
    """
    base = os.path.basename(src_path)
    m = re.match(r"^src_(\d{3})-(\d{3})s\.png$", base)
    if not m:
        raise ValueError(f"Unexpected filename format: {src_path}")
    start = int(m.group(1))
    end   = int(m.group(2))
    idx = f"{start:03d}-{end:03d}s"
    return idx, start, end

def task_to_dir(task_type: str) -> str:
    if task_type == "task1":
        return "describe"
    if task_type == "task2":
        return "compare"
    return "recall"

def ensure_task_dirs():
    """
    Ensure ALL required folders exist, even if some tasks are not present.
    """
    for td in ["describe", "compare", "recall"]:
        root = os.path.join(DEMO_ROOT, td)
        os.makedirs(os.path.join(root, "Original_image"), exist_ok=True)
        os.makedirs(os.path.join(root, "aois"), exist_ok=True)
        os.makedirs(os.path.join(root, "merged"), exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Font] Using LiberationMono


## Prompt, API Call, Render output image

In [6]:
# ============================================================
# 5) PROMPT ENGINEERING
# ============================================================
def build_prompt(task_type: str) -> str:
    base = """
You are analyzing a user's visual attention in an experiment.

Return:
Line 1: A short heading (<=10 words).
Line 2+: A focused paragraph (3-6 sentences).
No bullet points.
Be concise and emphasize key differences or important objects.
"""

    if task_type == "task1":
        return base + """
Task 1:
State what the participant is looking at.
Describe key visual features of the attended object.
"""
    if task_type == "task2":
        return base + """
Task 2:
State what the participant is doing (e.g., comparing two objects, evaluating
differences, inspecting features). Explain what specific objects or regions
are being compared and the key visual similarities and differences that are
likely being evaluated.
"""
    if task_type == "task3_1":
        return base + """
Task 3_1:
Participant is reviewing the photo.
Describe multiple important objects and their details.
"""
    if task_type == "task3_2":
        return base + """
Task 3_2:
Compare the REFERENCE image and CURRENT image.
Describe what is different.
"""
    if task_type == "task3":
        return base + """
Task 3:
Compare REFERENCE (Task1) and CURRENT image.
Explain what is missing and how gaze suggests the participant is reasoning.
"""
    return base

# ============================================================
# 6) GPT-4o CALL
# ============================================================
def call_gpt4o(prompt: str, image_paths: list) -> str:
    messages = [{
        "role": "user",
        "content": [{"type": "text", "text": prompt}]
    }]

    for path in image_paths:
        if not path or not os.path.exists(path):
            raise FileNotFoundError(f"Image input missing: {path}")
        messages[0]["content"].append({
            "type": "image_url",
            "image_url": {"url": f"data:image/png;base64,{encode_image(path)}"}
        })

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=600
    )
    return response.choices[0].message.content

# ============================================================
# 7) RENDER OUTPUT IMAGE (MERGED)
#    - AOI panel: left text label, right image (clear reference/current)
# ============================================================
def wrap_text(draw: ImageDraw.ImageDraw, text: str, font: ImageFont.ImageFont, max_width: int):
    words = text.split()
    lines = []
    current = ""
    for w in words:
        test = (current + " " + w).strip()
        if draw.textlength(test, font=font) <= max_width:
            current = test
        else:
            lines.append(current)
            current = w
    if current:
        lines.append(current)
    return lines

def paste_labeled_aoi(canvas: Image.Image,
                      draw: ImageDraw.ImageDraw,
                      label: str,
                      img_path: str,
                      x: int,
                      y: int,
                      row_w: int,
                      row_h: int,
                      label_w: int = 95,
                      gap: int = 10):
    """
    Draws: [label text][image]
    """
    # Label (left)
    draw.text((x, y + (row_h // 2) - 8), label, font=FONT_LABEL, fill=(120, 120, 120))

    # Image (right)
    img = Image.open(img_path).convert("RGB")
    img_w = row_w - label_w - gap
    img_h = row_h
    img = img.resize((img_w, img_h))
    canvas.paste(img, (x + label_w + gap, y))

def create_output_image(heat_path: str,
                        reference_crop: Optional[str],
                        current_crop: str,
                        text: str,
                        save_path: str,
                        title: str):
    CANVAS_W, CANVAS_H = 1200, 820
    TITLE_Y = 8
    LABEL_Y = 44
    TOP_Y = 60

    HEAT_W, HEAT_H = 640, 360
    RIGHT_X = 740

    BOX_X1, BOX_Y1 = 24, 470
    BOX_X2, BOX_Y2 = 1176, 800

    PAD_X = 18
    PAD_TOP = 14

    heat_img = Image.open(heat_path).convert("RGB").resize((HEAT_W, HEAT_H))

    canvas = Image.new("RGB", (CANVAS_W, CANVAS_H), (248, 248, 248))
    draw = ImageDraw.Draw(canvas)

    draw.text((24, TITLE_Y), f"Experiment Task: {title}", font=FONT_HEADER, fill=(20, 20, 20))
    draw.text((24, LABEL_Y), "User Gaze Context (Heatmap)", font=FONT_LABEL, fill=(90, 90, 90))
    draw.text((RIGHT_X, LABEL_Y), "Extracted AOI", font=FONT_LABEL, fill=(90, 90, 90))

    canvas.paste(heat_img, (24, TOP_Y))

    # AOI panel layout
    AOI_ROW_W = 420
    AOI_ROW_H = 170

    if reference_crop and os.path.exists(reference_crop):
        # Two rows: Reference then Current (label left, image right)
        paste_labeled_aoi(canvas, draw, "Reference", reference_crop, RIGHT_X, TOP_Y, AOI_ROW_W, AOI_ROW_H)
        paste_labeled_aoi(canvas, draw, "Current",   current_crop,   RIGHT_X, TOP_Y + AOI_ROW_H + 20, AOI_ROW_W, AOI_ROW_H)
    else:
        # One row: Current only
        paste_labeled_aoi(canvas, draw, "Current", current_crop, RIGHT_X, TOP_Y, AOI_ROW_W, 260)

    # Bottom text box
    draw.rounded_rectangle(
        [BOX_X1, BOX_Y1, BOX_X2, BOX_Y2],
        radius=18,
        fill=(255, 255, 255),
        outline=(220, 220, 220),
        width=2
    )
    draw.text((BOX_X1 + PAD_X, BOX_Y1 + PAD_TOP), "AI Attention-Aware Output", font=FONT_LABEL, fill=(25, 90, 160))

    text = sanitize_model_text(text)
    lines = text.split("\n", 1)
    heading = (lines[0].strip() if lines and lines[0].strip() else "Attention Summary")
    body = (lines[1].strip() if len(lines) > 1 else "")

    heading_x = BOX_X1 + PAD_X
    heading_y = BOX_Y1 + 52
    draw.text((heading_x, heading_y), heading, font=FONT_HEADER, fill=(30, 30, 30))

    body_x = BOX_X1 + PAD_X
    body_y = heading_y + 50

    body_max_w = (BOX_X2 - BOX_X1) - 2 * PAD_X
    body_max_h = (BOX_Y2 - body_y) - 14

    wrapped = wrap_text(draw, body, FONT_BODY, body_max_w)

    line_h = 24
    max_lines = max(0, body_max_h // line_h)

    truncated = len(wrapped) > max_lines
    wrapped = wrapped[:max_lines]

    if truncated and max_lines > 0:
        last = wrapped[-1]
        ell = "..."
        while last and draw.textlength(last + ell, font=FONT_BODY) > body_max_w:
            last = last[:-1].rstrip()
        wrapped[-1] = (last + ell) if last else ell

    y = body_y
    for ln in wrapped:
        draw.text((body_x, y), ln, font=FONT_BODY, fill=(20, 20, 20))
        y += line_h

    canvas.save(save_path)

##  Main Loop

In [7]:
# ============================================================
# 8) MAIN PIPELINE
# ============================================================
ensure_task_dirs()
print("Starting NEAR Pipeline...")

for folder in ALL_FOLDERS:
    frames_dir = os.path.join(BASE_PATH, folder, "frames")
    if not os.path.exists(frames_dir):
        continue

    task_type = parse_task_type(folder)
    task_dir = task_to_dir(task_type)
    prompt = build_prompt(task_type)

    out_task_root = os.path.join(DEMO_ROOT, task_dir)
    out_original = os.path.join(out_task_root, "Original_image")
    out_aois = os.path.join(out_task_root, "aois")
    out_merged = os.path.join(out_task_root, "merged")

    os.makedirs(out_original, exist_ok=True)
    os.makedirs(out_aois, exist_ok=True)
    os.makedirs(out_merged, exist_ok=True)

    sources = glob.glob(os.path.join(frames_dir, "src_*.png"))
    sources = [p for p in sources if not p.endswith("_crop.png")]

    def sort_key(p: str) -> int:
        _, start, _ = parse_src_filename(p)
        return start

    sources = sorted(sources, key=sort_key)

    print(f"\nProcessing {folder} ({len(sources)} images) -> {task_dir}")

    responses: Dict[str, str] = {}

    for i, src_path in enumerate(tqdm(sources, desc=folder), start=1):
        idx, start, end = parse_src_filename(src_path)  # idx: 000-003s
        seq = f"{i:03d}"                                # 001, 002, ...

        heat_path = os.path.join(frames_dir, f"heat_{idx}.png")
        current_crop = prefer_crop(frames_dir, idx)

        if not os.path.exists(heat_path):
            raise FileNotFoundError(f"Missing heatmap: {heat_path}")
        if not current_crop or not os.path.exists(current_crop):
            raise FileNotFoundError(f"Missing AOI crop for idx={idx} in {frames_dir}")

        reference_crop = None
        # Task3_2: reference from _3_1
        if task_type == "task3_2":
            ref_folder = get_reference_folder(folder, "_3_1")
            ref_frames = os.path.join(BASE_PATH, ref_folder, "frames")
            reference_crop = prefer_crop(ref_frames, idx)

        # Task3: reference from _1 (task1)
        if task_type == "task3":
            ref_folder = get_reference_folder(folder, "_1")
            ref_frames = os.path.join(BASE_PATH, ref_folder, "frames")
            reference_crop = prefer_crop(ref_frames, idx)

        # ----------------------------------------------------
        # Save outputs (ALL required folders + correct naming)
        # ----------------------------------------------------
        # Per your spec: heatmap stored in Original_image/
        shutil.copy2(heat_path, os.path.join(out_original, f"{seq}.png"))

        # AOI storing rules:
        # - If reference exists: save both -> 002_ref.png and 002_current.png
        # - Else: save only current -> 002.png
        if reference_crop and os.path.exists(reference_crop):
            shutil.copy2(reference_crop, os.path.join(out_aois, f"{seq}_ref.png"))
            shutil.copy2(current_crop,   os.path.join(out_aois, f"{seq}_current.png"))
        else:
            shutil.copy2(current_crop, os.path.join(out_aois, f"{seq}.png"))

        # LLM inputs
        image_inputs = [src_path]
        if reference_crop and os.path.exists(reference_crop):
            image_inputs.append(reference_crop)
        image_inputs.append(current_crop)

        result = call_gpt4o(prompt, image_inputs)
        responses[seq] = result

        # Merged image
        title = f"{task_dir} | {task_type} | idx {idx} | {seq}"
        merged_save_path = os.path.join(out_merged, f"{seq}.png")

        create_output_image(
            heat_path=heat_path,
            reference_crop=reference_crop if (reference_crop and os.path.exists(reference_crop)) else None,
            current_crop=current_crop,
            text=result,
            save_path=merged_save_path,
            title=title
        )

    # Save responses.json
    json_path = os.path.join(out_task_root, "responses.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(responses, f, ensure_ascii=False, indent=2)

print("\nFinished.")
print("\n=== Output Directory Structure ===")

for task_name in ["describe", "compare", "recall"]:
    task_root = os.path.join(DEMO_ROOT, task_name)
    print(f"\nTask: {task_name}")
    print(" Root:           ", task_root)
    print(" Original_image: ", os.path.join(task_root, "Original_image"))
    print(" AOIs:           ", os.path.join(task_root, "aois"))
    print(" Merged:         ", os.path.join(task_root, "merged"))
    print(" Responses JSON: ", os.path.join(task_root, "responses.json"))

Starting NEAR Pipeline...

Processing Ayu_1 (20 images) -> describe


Ayu_1:   0%|          | 0/20 [00:00<?, ?it/s]


Processing Ayu_2 (14 images) -> compare


Ayu_2:   0%|          | 0/14 [00:00<?, ?it/s]


Processing Ayu_3 (26 images) -> recall


Ayu_3:   0%|          | 0/26 [00:00<?, ?it/s]


Finished.

=== Output Directory Structure ===

Task: describe
 Root:            /content/drive/My Drive/AI_event_demo/demo_data/describe
 Original_image:  /content/drive/My Drive/AI_event_demo/demo_data/describe/Original_image
 AOIs:            /content/drive/My Drive/AI_event_demo/demo_data/describe/aois
 Merged:          /content/drive/My Drive/AI_event_demo/demo_data/describe/merged
 Responses JSON:  /content/drive/My Drive/AI_event_demo/demo_data/describe/responses.json

Task: compare
 Root:            /content/drive/My Drive/AI_event_demo/demo_data/compare
 Original_image:  /content/drive/My Drive/AI_event_demo/demo_data/compare/Original_image
 AOIs:            /content/drive/My Drive/AI_event_demo/demo_data/compare/aois
 Merged:          /content/drive/My Drive/AI_event_demo/demo_data/compare/merged
 Responses JSON:  /content/drive/My Drive/AI_event_demo/demo_data/compare/responses.json

Task: recall
 Root:            /content/drive/My Drive/AI_event_demo/demo_data/recall
 Origi

# Run all tester
output path: /content/drive/My Drive/AI_event_demo/Attention-Aware_Output

```text
Attention-Aware_Output/
├── AT/
│   ├── describe/
│   ├── compare/
│   └── recall/
├── Ayu/
│   ├── describe/
│   ├── compare/
│   └── recall/
├── JC/
│   ├── describe/
│   ├── compare/
│   └── recall/
```

In [4]:
import os
import re
import glob
import json
import shutil
import base64
from typing import Optional, Tuple, Dict

from tqdm.auto import tqdm
from PIL import Image, ImageDraw, ImageFont
from openai import OpenAI
from google.colab import drive, userdata

# ============================================================
# 1) AUTHENTICATION
# ============================================================
OPENAI_API_KEY = userdata.get("OPENAI_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

drive.mount("/content/drive")

# ============================================================
# 2) PATH CONFIG
# ============================================================
BASE_PATH = "/content/drive/My Drive/AI_event_demo/NEAR_Experiment_Design_Output/"
OUTPUT_ROOT = "/content/drive/My Drive/AI_event_demo/Attention-Aware_Output"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

TASK_GROUPS = [
    ["AT_1", "AT_2", "AT_3_1", "AT_3_2"],
    ["Ayu_1", "Ayu_2", "Ayu_3"],
    ["JC_1", "JC_2", "JC_3_1", "JC_3_2"],
    ["KC_1", "KC_2", "KC_3_1", "KC_3_2"],
    ["LKH_1", "LKH_2", "LKH_3_1", "LKH_3_2"],
    ["SYH_1_simple", "SYH_2_simple", "SYH_3_1_simple", "SYH_3_2_simple"],
    ["YL_1", "YL_2", "YL_3"],
]
ALL_FOLDERS = [f for g in TASK_GROUPS for f in g]

# ============================================================
# 3) FONT SETUP
# ============================================================
def load_fonts():
    mono_regular = "/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf"
    mono_bold    = "/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf"

    if os.path.exists(mono_regular) and os.path.exists(mono_bold):
        header = ImageFont.truetype(mono_bold, 18)
        label  = ImageFont.truetype(mono_bold, 13)
        body   = ImageFont.truetype(mono_regular, 13)
        print("[Font] Using LiberationMono")
        return header, label, body

    print("[Font] LiberationMono not found. Falling back.")
    return ImageFont.load_default(), ImageFont.load_default(), ImageFont.load_default()

FONT_HEADER, FONT_LABEL, FONT_BODY = load_fonts()

# ============================================================
# 4) HELPERS
# ============================================================
def encode_image(path: str) -> str:
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def parse_task_type(folder_name: str) -> str:
    name = folder_name.replace("_simple", "")
    if "_3_1" in name:
        return "task3_1"
    if "_3_2" in name:
        return "task3_2"
    if name.endswith("_1"):
        return "task1"
    if name.endswith("_2"):
        return "task2"
    if name.endswith("_3"):
        return "task3"
    return "unknown"

def get_group_name(folder: str) -> str:
    return folder.split("_")[0]

def get_reference_folder(folder: str, suffix: str) -> str:
    return f"{get_group_name(folder)}{suffix}"

def prefer_crop(frames_dir: str, idx: str) -> Optional[str]:
    """
    Prefer src_{idx}_crop.png, else heat_{idx}_crop.png.
    idx example: '000-003s'
    """
    src_crop = os.path.join(frames_dir, f"src_{idx}_crop.png")
    heat_crop = os.path.join(frames_dir, f"heat_{idx}_crop.png")
    if os.path.exists(src_crop):
        return src_crop
    if os.path.exists(heat_crop):
        return heat_crop
    return None

def sanitize_model_text(text: str) -> str:
    if not text:
        return ""
    text = re.sub(r"^\s*#{1,6}\s*", "", text, flags=re.MULTILINE)
    text = text.replace("**", "").replace("*", "")
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def parse_src_filename(src_path: str) -> Tuple[str, int, int]:
    """
    Enforce: src_000-003s.png
    Returns:
      idx: '000-003s'
      start: 0
      end: 3
    """
    base = os.path.basename(src_path)
    m = re.match(r"^src_(\d{3})-(\d{3})s\.png$", base)
    if not m:
        raise ValueError(f"Unexpected filename format: {src_path}")
    start = int(m.group(1))
    end   = int(m.group(2))
    idx = f"{start:03d}-{end:03d}s"
    return idx, start, end

def task_to_dir(task_type: str) -> str:
    if task_type == "task1":
        return "describe"
    if task_type == "task2":
        return "compare"
    return "recall"

def ensure_task_dirs(subject_root: str):
    """
    Ensure task output folders exist under a subject root:
    subject_root/{describe,compare,recall}/{Original_image,aois,merged}
    """
    for td in ["describe", "compare", "recall"]:
        root = os.path.join(subject_root, td)
        os.makedirs(os.path.join(root, "Original_image"), exist_ok=True)
        os.makedirs(os.path.join(root, "aois"), exist_ok=True)
        os.makedirs(os.path.join(root, "merged"), exist_ok=True)


# ============================================================
# 5) PROMPT ENGINEERING
# ============================================================
def build_prompt(task_type: str) -> str:
    base = """
You are analyzing a user's visual attention in an experiment.

Return:
Line 1: A short heading (<=10 words).
Line 2+: A focused paragraph (3-6 sentences).
No bullet points.
Be concise and emphasize key differences or important objects.
"""

    if task_type == "task1":
        return base + """
Task 1:
State what the participant is looking at.
Describe key visual features of the attended object.
"""
    if task_type == "task2":
        return base + """
Task 2:
State what the participant is doing (e.g., comparing two objects, evaluating
differences, inspecting features). Explain what specific objects or regions
are being compared and the key visual similarities and differences that are
likely being evaluated.
"""
    if task_type == "task3_1":
        return base + """
Task 3_1:
Participant is reviewing the photo.
Describe multiple important objects and their details.
"""
    if task_type == "task3_2":
        return base + """
Task 3_2:
Compare the REFERENCE image and CURRENT image.
Describe what is different.
"""
    if task_type == "task3":
        return base + """
Task 3:
Compare REFERENCE (Task1) and CURRENT image.
Explain what is missing and how gaze suggests the participant is reasoning.
"""
    return base

# ============================================================
# 6) GPT-4o CALL
# ============================================================
def call_gpt4o(prompt: str, image_paths: list) -> str:
    messages = [{
        "role": "user",
        "content": [{"type": "text", "text": prompt}]
    }]

    for path in image_paths:
        if not path or not os.path.exists(path):
            raise FileNotFoundError(f"Image input missing: {path}")
        messages[0]["content"].append({
            "type": "image_url",
            "image_url": {"url": f"data:image/png;base64,{encode_image(path)}"}
        })

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=600
    )
    return response.choices[0].message.content

# ============================================================
# 7) RENDER OUTPUT IMAGE (MERGED)
#    - AOI panel: left text label, right image (clear reference/current)
# ============================================================
def wrap_text(draw: ImageDraw.ImageDraw, text: str, font: ImageFont.ImageFont, max_width: int):
    words = text.split()
    lines = []
    current = ""
    for w in words:
        test = (current + " " + w).strip()
        if draw.textlength(test, font=font) <= max_width:
            current = test
        else:
            lines.append(current)
            current = w
    if current:
        lines.append(current)
    return lines

def paste_labeled_aoi(canvas: Image.Image,
                      draw: ImageDraw.ImageDraw,
                      label: str,
                      img_path: str,
                      x: int,
                      y: int,
                      row_w: int,
                      row_h: int,
                      label_w: int = 95,
                      gap: int = 10):
    """
    Draws: [label text][image]
    """
    # Label (left)
    draw.text((x, y + (row_h // 2) - 8), label, font=FONT_LABEL, fill=(120, 120, 120))

    # Image (right)
    img = Image.open(img_path).convert("RGB")
    img_w = row_w - label_w - gap
    img_h = row_h
    img = img.resize((img_w, img_h))
    canvas.paste(img, (x + label_w + gap, y))

def create_output_image(heat_path: str,
                        reference_crop: Optional[str],
                        current_crop: str,
                        text: str,
                        save_path: str,
                        title: str):
    CANVAS_W, CANVAS_H = 1200, 820
    TITLE_Y = 8
    LABEL_Y = 44
    TOP_Y = 60

    HEAT_W, HEAT_H = 640, 360
    RIGHT_X = 740

    BOX_X1, BOX_Y1 = 24, 470
    BOX_X2, BOX_Y2 = 1176, 800

    PAD_X = 18
    PAD_TOP = 14

    heat_img = Image.open(heat_path).convert("RGB").resize((HEAT_W, HEAT_H))

    canvas = Image.new("RGB", (CANVAS_W, CANVAS_H), (248, 248, 248))
    draw = ImageDraw.Draw(canvas)

    draw.text((24, TITLE_Y), f"Experiment Task: {title}", font=FONT_HEADER, fill=(20, 20, 20))
    draw.text((24, LABEL_Y), "User Gaze Context (Heatmap)", font=FONT_LABEL, fill=(90, 90, 90))
    draw.text((RIGHT_X, LABEL_Y), "Extracted AOI", font=FONT_LABEL, fill=(90, 90, 90))

    canvas.paste(heat_img, (24, TOP_Y))

    # AOI panel layout
    AOI_ROW_W = 420
    AOI_ROW_H = 170

    if reference_crop and os.path.exists(reference_crop):
        # Two rows: Reference then Current (label left, image right)
        paste_labeled_aoi(canvas, draw, "Reference", reference_crop, RIGHT_X, TOP_Y, AOI_ROW_W, AOI_ROW_H)
        paste_labeled_aoi(canvas, draw, "Current",   current_crop,   RIGHT_X, TOP_Y + AOI_ROW_H + 20, AOI_ROW_W, AOI_ROW_H)
    else:
        # One row: Current only
        paste_labeled_aoi(canvas, draw, "Current", current_crop, RIGHT_X, TOP_Y, AOI_ROW_W, 260)

    # Bottom text box
    draw.rounded_rectangle(
        [BOX_X1, BOX_Y1, BOX_X2, BOX_Y2],
        radius=18,
        fill=(255, 255, 255),
        outline=(220, 220, 220),
        width=2
    )
    draw.text((BOX_X1 + PAD_X, BOX_Y1 + PAD_TOP), "AI Attention-Aware Output", font=FONT_LABEL, fill=(25, 90, 160))

    text = sanitize_model_text(text)
    lines = text.split("\n", 1)
    heading = (lines[0].strip() if lines and lines[0].strip() else "Attention Summary")
    body = (lines[1].strip() if len(lines) > 1 else "")

    heading_x = BOX_X1 + PAD_X
    heading_y = BOX_Y1 + 52
    draw.text((heading_x, heading_y), heading, font=FONT_HEADER, fill=(30, 30, 30))

    body_x = BOX_X1 + PAD_X
    body_y = heading_y + 50

    body_max_w = (BOX_X2 - BOX_X1) - 2 * PAD_X
    body_max_h = (BOX_Y2 - body_y) - 14

    wrapped = wrap_text(draw, body, FONT_BODY, body_max_w)

    line_h = 24
    max_lines = max(0, body_max_h // line_h)

    truncated = len(wrapped) > max_lines
    wrapped = wrapped[:max_lines]

    if truncated and max_lines > 0:
        last = wrapped[-1]
        ell = "..."
        while last and draw.textlength(last + ell, font=FONT_BODY) > body_max_w:
            last = last[:-1].rstrip()
        wrapped[-1] = (last + ell) if last else ell

    y = body_y
    for ln in wrapped:
        draw.text((body_x, y), ln, font=FONT_BODY, fill=(20, 20, 20))
        y += line_h

    canvas.save(save_path)

# ============================================================
# 8) MAIN PIPELINE
# ============================================================

print("Starting NEAR Pipeline...")

for folder in ALL_FOLDERS:
    frames_dir = os.path.join(BASE_PATH, folder, "frames")
    if not os.path.exists(frames_dir):
        continue

    task_type = parse_task_type(folder)
    task_dir = task_to_dir(task_type)
    prompt = build_prompt(task_type)

    # Subject folder (AT, Ayu, JC, ...)
    subject_name = get_group_name(folder)
    subject_root = os.path.join(OUTPUT_ROOT, subject_name)

    # Ensure subject-level task folders exist
    ensure_task_dirs(subject_root)

    # Output under subject/{describe|compare|recall}
    out_task_root = os.path.join(subject_root, task_dir)

    out_original = os.path.join(out_task_root, "Original_image")
    out_aois = os.path.join(out_task_root, "aois")
    out_merged = os.path.join(out_task_root, "merged")

    os.makedirs(out_original, exist_ok=True)
    os.makedirs(out_aois, exist_ok=True)
    os.makedirs(out_merged, exist_ok=True)

    sources = glob.glob(os.path.join(frames_dir, "src_*.png"))
    sources = [p for p in sources if not p.endswith("_crop.png")]

    def sort_key(p: str) -> int:
        _, start, _ = parse_src_filename(p)
        return start

    sources = sorted(sources, key=sort_key)

    print(f"\nProcessing {folder} ({len(sources)} images) -> {task_dir}")

    responses: Dict[str, str] = {}

    for i, src_path in enumerate(tqdm(sources, desc=folder), start=1):
        idx, start, end = parse_src_filename(src_path)  # idx: 000-003s
        seq = f"{i:03d}"                                # 001, 002, ...

        heat_path = os.path.join(frames_dir, f"heat_{idx}.png")
        current_crop = prefer_crop(frames_dir, idx)

        if not os.path.exists(heat_path):
            raise FileNotFoundError(f"Missing heatmap: {heat_path}")
        if not current_crop or not os.path.exists(current_crop):
            raise FileNotFoundError(f"Missing AOI crop for idx={idx} in {frames_dir}")

        reference_crop = None
        # Task3_2: reference from _3_1
        if task_type == "task3_2":
            ref_folder = get_reference_folder(folder, "_3_1")
            ref_frames = os.path.join(BASE_PATH, ref_folder, "frames")
            reference_crop = prefer_crop(ref_frames, idx)

        # Task3: reference from _1 (task1)
        if task_type == "task3":
            ref_folder = get_reference_folder(folder, "_1")
            ref_frames = os.path.join(BASE_PATH, ref_folder, "frames")
            reference_crop = prefer_crop(ref_frames, idx)

        # ----------------------------------------------------
        # Save outputs (ALL required folders + correct naming)
        # ----------------------------------------------------
        # Per your spec: heatmap stored in Original_image/
        shutil.copy2(heat_path, os.path.join(out_original, f"{seq}.png"))

        # AOI storing rules:
        # - If reference exists: save both -> 002_ref.png and 002_current.png
        # - Else: save only current -> 002.png
        if reference_crop and os.path.exists(reference_crop):
            shutil.copy2(reference_crop, os.path.join(out_aois, f"{seq}_ref.png"))
            shutil.copy2(current_crop,   os.path.join(out_aois, f"{seq}_current.png"))
        else:
            shutil.copy2(current_crop, os.path.join(out_aois, f"{seq}.png"))

        # LLM inputs
        image_inputs = [src_path]
        if reference_crop and os.path.exists(reference_crop):
            image_inputs.append(reference_crop)
        image_inputs.append(current_crop)

        result = call_gpt4o(prompt, image_inputs)
        responses[seq] = result

        # Merged image
        title = f"{task_dir} | {task_type} | idx {idx} | {seq}"
        merged_save_path = os.path.join(out_merged, f"{seq}.png")

        create_output_image(
            heat_path=heat_path,
            reference_crop=reference_crop if (reference_crop and os.path.exists(reference_crop)) else None,
            current_crop=current_crop,
            text=result,
            save_path=merged_save_path,
            title=title
        )

    # Save responses.json
    json_path = os.path.join(out_task_root, "responses.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(responses, f, ensure_ascii=False, indent=2)

print("\nFinished.")
print("\n=== Output Directory Structure ===")

subjects = sorted({get_group_name(f) for f in ALL_FOLDERS})

for subj in subjects:
    subject_root = os.path.join(OUTPUT_ROOT, subj)
    print(f"\nSubject: {subj}")
    print(" Root:           ", subject_root)

    for task_name in ["describe", "compare", "recall"]:
        task_root = os.path.join(subject_root, task_name)
        print(f"  Task: {task_name}")
        print("   Original_image: ", os.path.join(task_root, "Original_image"))
        print("   AOIs:           ", os.path.join(task_root, "aois"))
        print("   Merged:         ", os.path.join(task_root, "merged"))
        print("   Responses JSON: ", os.path.join(task_root, "responses.json"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Font] Using LiberationMono
Starting NEAR Pipeline...

Processing AT_1 (24 images) -> describe


AT_1:   0%|          | 0/24 [00:00<?, ?it/s]


Processing AT_2 (10 images) -> compare


AT_2:   0%|          | 0/10 [00:00<?, ?it/s]


Processing AT_3_1 (7 images) -> recall


...


Processing YL_2 (9 images) -> compare


YL_2:   0%|          | 0/9 [00:00<?, ?it/s]


Processing YL_3 (8 images) -> recall


YL_3:   0%|          | 0/8 [00:00<?, ?it/s]


Finished.

=== Output Directory Structure ===

Subject: AT
 Root:            /content/drive/My Drive/AI_event_demo/Attention-Aware_Output/AT
  Task: describe
   Original_image:  /content/drive/My Drive/AI_event_demo/Attention-Aware_Output/AT/describe/Original_image
   AOIs:            /content/drive/My Drive/AI_event_demo/Attention-Aware_Output/AT/describe/aois
   Merged:          /content/drive/My Drive/AI_event_demo/Attention-Aware_Output/AT/describe/merged
   Responses JSON:  /content/drive/My Drive/AI_event_demo/Attention-Aware_Output/AT/describe/responses.json
  Task: compare
   Original_image:  /content/drive/My Drive/AI_event_demo/Attention-Aware_Output/AT/compare/Original_image
   AOIs:            /content/drive/My Drive/AI_event_demo/Attention-Aware_Output/AT/compare/aois
   Merged:          /content/drive/My Drive/AI_event_demo/Attention-Aware_Output/AT/compare/merged
   Responses JSON:  /content/drive/My Drive/AI_event_demo/Attention-Aware_Output/AT/compare/responses.json
 